In [109]:
import pandas as pd
import numpy as np
import pulp

In [110]:
from functions import get_current_weekday, calculate_nfl_week, get_next_sunday, get_current_year

In [111]:
day = get_current_weekday()

In [112]:
date_string = get_next_sunday(day)

In [113]:
week = calculate_nfl_week(date_string)

In [114]:
season = get_current_year()

In [115]:
site_for_pred = input('Do you want to generate lineups for FanDuel or DraftKings? ').upper()
if site_for_pred == 'FD':
    df = pd.read_csv('FD_predictions_' + str(season) + '_' + str(week) + '.csv')
else: 
    df = pd.read_csv('DK_predictions_' + str(season) + '_' + str(week) + '.csv')

Do you want to generate lineups for FanDuel or DraftKings?  DK


In [116]:
# dk_preds['Value'] = np.round((dk_preds['median_pred'] / dk_preds['salary']) * 1000, 3)

In [117]:
# dk_preds.sort_values(by = ['Value'], ascending = False)

In [118]:
# prob = pulp.LpProblem('DK_NFL_weekly', pulp.LpMaximize)
# player_vars = [pulp.LpVariable(f'player_{row.name}', cat='Binary') for row in df.itertuples()]

In [119]:
# total assigned players constraint
# prob += pulp.lpSum(player_var for player_var in player_vars) == 9
# # total salary constraint
# prob += pulp.lpSum(df.salary.iloc[i] * player_vars[i] for i in range(len(df))) <= 50000
# # create a helper function to return the number of players assigned each position
# def get_position_sum(player_vars, df, position):
#     return pulp.lpSum([player_vars[i] * (position in df['position'].iloc[i]) for i in range(len(df))])
    
# # for QB and DST, require 1 of each in the lineup
# prob += get_position_sum(player_vars, df, 'QB') == 1
# prob += get_position_sum(player_vars, df, 'D') == 1
# prob += get_position_sum(player_vars, df, 'TE') == 1
# # to account for the FLEX position, we allow additional selections of the 3 FLEX-eligible positions: RB, WR, TE
# prob += get_position_sum(player_vars, df, 'RB') >= 2
# prob += get_position_sum(player_vars, df, 'WR') >= 3


In [120]:
# prob += pulp.lpSum([df.max_pred.iloc[i] * player_vars[i] for i in range(len(df))])

In [121]:
# solve and print the status
# prob.solve()
# print(pulp.LpStatus[prob.status])

In [122]:
# total_salary_used = 0
# mean_pred_points = 0
# for i in range(len(df)):
#     if player_vars[i].value() == 1:
#         row = df.iloc[i]
#         print(row['position'], row['name'], row.team, row.salary, row.median_pred)
#         total_salary_used += row.salary
#         mean_pred_points += row.median_pred
# mean_pred_points /= 9  # divide by total players in roster to get a mean

# print(f"Total salary used: {total_salary_used}")
# print(f"Total predicted points: {mean_pred_points * 9}")  # Multiply mean by 9 to get total points


In [123]:
# exclude_list = []
# lock_list = []

In [124]:
########################## LINEUP GENERATION CODE. IT WORKS. EXPERIMENTING WITH LOCKING PLAYERS IN NEXT CELL #######
# exclude = input('Do you want to exclude any players from lineups? ')
# while exclude == 'y':
#     player = input('Which player? ')
#     exclude_list.append(player)
#     exclude = input('Any other players? ')

# if site_for_pred == 'FD':
#     cap = 60000
# else:
#     cap = 50000

# # Define the lineup generation function with a salary cap parameter
# def generate_lineup(df, salary_cap, excluded_players=exclude_list):
#     if excluded_players is None:
#         excluded_players = []

#     # Create the pulp problem
#     prob = pulp.LpProblem('NFL_weekly', pulp.LpMaximize)

#     # Create variables for each player indicating whether they are included in the lineup
#     player_vars = [pulp.LpVariable(f'player_{row.name}', cat='Binary') for row in df.itertuples()]

#     # Total assigned players constraint
#     prob += pulp.lpSum(player_var for player_var in player_vars) == 9

#     # Total salary constraint using the provided salary cap
#     prob += pulp.lpSum(df.salary.iloc[i] * player_vars[i] for i in range(len(df))) <= salary_cap

#     # Create a helper function to return the number of players assigned each position
#     def get_position_sum(player_vars, df, position):
#         return pulp.lpSum([player_vars[i] * (position in df['position'].iloc[i]) for i in range(len(df))])

#     # Position constraints: 1 QB, 1 Defense, 1 TE, 2 RBs, 3 WRs, 1 FLEX (RB, WR, or TE)
#     prob += get_position_sum(player_vars, df, 'QB') == 1
#     prob += get_position_sum(player_vars, df, 'D') == 1
#     prob += get_position_sum(player_vars, df, 'TE') == 1
#     prob += get_position_sum(player_vars, df, 'RB') >= 2
#     prob += get_position_sum(player_vars, df, 'WR') >= 3

#     # Objective: Maximize total predicted points
#     prob += pulp.lpSum([df.max_pred.iloc[i] * player_vars[i] for i in range(len(df))])

#     # Exclude specific players
#     for i, row in df.iterrows():
#         if row['name'] in excluded_players:
#             prob += player_vars[i] == 0  # Prevent this player from being selected

#     # Solve the problem and print the status
#     prob.solve()
#     print(pulp.LpStatus[prob.status])

#     total_salary_used = 0
#     mean_pred_points = 0
#     lineup = []

#     # Gather lineup details
#     for i in range(len(df)):
#         if player_vars[i].value() == 1:
#             row = df.iloc[i]
#             lineup.append((row['position'], row['name'], row.team, row.salary, row.median_pred))
#             total_salary_used += row.salary
#             mean_pred_points += row.median_pred

#     mean_pred_points /= 9  # Divide by total players in roster to get a mean

#     # Print the lineup details
#     for player in lineup:
#         print(player)

#     print(f"Total salary used: {total_salary_used}")
#     print(f"Total predicted points: {mean_pred_points * 9}")  # Multiply mean by 9 to get total points

#     return lineup

# # Example of how to call the function for DraftKings

# generate_lineup(df, cap, exclude_list)

# generate = input('Do you want to generate any more lineups? ')
# while generate == 'y':
#     site_for_pred = input('Which site? ')
#     if site_for_pred == 'FD':
#         df = pd.read_csv('FD_predictions_' + str(season) + '_' + str(week) + '.csv')
#         cap = 60000
#     else: 
#         df = pd.read_csv('DK_predictions_' + str(season) + '_' + str(week) + '.csv')
#         cap = 50000
#     exclude = input('Do you want to exclude any players from lineups? ')
#     while exclude == 'y':
#         player = input('Which player? ')
#         exclude_list.append(player)
#         exclude = input('Any other players? ')
#     generate_lineup(df, cap, exclude_list)
#     generate = input('Do you want to generate any more lineups? ')

In [125]:
##################### CODE THAT ALLOWS PLAYER LOCK ###########################
# exclude_list = []
# lock_list = []

# #Prompt user for players to lock into the lineup
# lock = input('Do you want to lock any players into the lineup? ')
# while lock == 'y':
#     player = input('Which player? ')
#     lock_list.append(player)
#     lock = input('Any other players? ')

# # Prompt user for players to exclude from the lineup
# exclude = input('Do you want to exclude any players from lineups? ')
# while exclude == 'y':
#     player = input('Which player? ')
#     exclude_list.append(player)
#     exclude = input('Any other players? ')

# if site_for_pred == 'FD':
#     cap = 60000
# else:
#     cap = 50000

# # Define the lineup generation function with a salary cap parameter and lock/exclude lists
# def generate_lineup(df, salary_cap, excluded_players=exclude_list, locked_players=lock_list):
#     if excluded_players is None:
#         excluded_players = []
#     if locked_players is None:
#         locked_players = []

#     # Create the pulp problem
#     prob = pulp.LpProblem('NFL_weekly', pulp.LpMaximize)

#     # Create variables for each player indicating whether they are included in the lineup
#     player_vars = [pulp.LpVariable(f'player_{row.name}', cat='Binary') for row in df.itertuples()]

#     # Total assigned players constraint
#     prob += pulp.lpSum(player_var for player_var in player_vars) == 9

#     # Total salary constraint using the provided salary cap
#     prob += pulp.lpSum(df.salary.iloc[i] * player_vars[i] for i in range(len(df))) <= salary_cap

#     # Create a helper function to return the number of players assigned each position
#     def get_position_sum(player_vars, df, position):
#         return pulp.lpSum([player_vars[i] * (position in df['position'].iloc[i]) for i in range(len(df))])

#     # Position constraints: 1 QB, 1 Defense, 1 TE, 2 RBs, 3 WRs, 1 FLEX (RB, WR, or TE)
#     prob += get_position_sum(player_vars, df, 'QB') == 1
#     prob += get_position_sum(player_vars, df, 'D') == 1
#     prob += get_position_sum(player_vars, df, 'TE') == 1
#     prob += get_position_sum(player_vars, df, 'RB') >= 2
#     prob += get_position_sum(player_vars, df, 'WR') >= 3

#     # Objective: Maximize total predicted points
#     prob += pulp.lpSum([df.max_pred.iloc[i] * player_vars[i] for i in range(len(df))])

#     # Exclude specific players
#     for i, row in df.iterrows():
#         if row['name'] in excluded_players:
#             prob += player_vars[i] == 0  # Prevent this player from being selected

#     # Lock specific players into the lineup
#     for i, row in df.iterrows():
#         if row['name'] in locked_players:
#             prob += player_vars[i] == 1  # Force this player to be selected

#     # Solve the problem and print the status
#     prob.solve()
#     print(pulp.LpStatus[prob.status])

#     total_salary_used = 0
#     mean_pred_points = 0
#     lineup = []

#     # Gather lineup details
#     for i in range(len(df)):
#         if player_vars[i].value() == 1:
#             row = df.iloc[i]
#             lineup.append((row['position'], row['name'], row.team, row.salary, row.median_pred))
#             total_salary_used += row.salary
#             mean_pred_points += row.median_pred

#     mean_pred_points /= 9  # Divide by total players in roster to get a mean

#     # Print the lineup details
#     for player in lineup:
#         print(player)

#     print(f"Total salary used: {total_salary_used}")
#     print(f"Total predicted points: {mean_pred_points * 9}")  # Multiply mean by 9 to get total points

#     return lineup

# # Example of how to call the function for DraftKings

# generate_lineup(df, cap, exclude_list, lock_list)

# generate = input('Do you want to generate any more lineups? ')
# while generate == 'y':
#     site_for_pred = input('Which site? ')
#     if site_for_pred == 'FD':
#         df = pd.read_csv('FD_predictions_' + str(season) + '_' + str(week) + '.csv')
#         cap = 60000
#     else: 
#         df = pd.read_csv('DK_predictions_' + str(season) + '_' + str(week) + '.csv')
#         cap = 50000
#     exclude = input('Do you want to exclude any players from lineups? ')
#     while exclude == 'y':
#         player = input('Which player? ')
#         exclude_list.append(player)
#         exclude = input('Any other players? ')
#     lock = input('Do you want to lock any players into the lineup? ')
#     while lock == 'y':
#         player = input('Which player? ')
#         lock_list.append(player)
#         lock = input('Any other players? ')
#     generate_lineup(df, cap, exclude_list, lock_list)
#     generate = input('Do you want to generate any more lineups? ')


In [126]:
##################### TRYING CODE THAT ALLOWS PLAYER LOCK. RESETTING OF LISTS AND TEAM EXCLUDE ###########################
exclude_list = []
lock_list = []
exclude_teams = []

# Prompt user for players to lock into the lineup
lock = input('Do you want to lock any players into the lineup? ').lower()
while lock == 'y':
    player = input('Which player? ')
    lock_list.append(player)
    lock = input('Any other players? ').lower()

# Prompt user for players to exclude from the lineup
exclude = input('Do you want to exclude any players from lineups? ').lower()
while exclude == 'y':
    player = input('Which player? ')
    exclude_list.append(player)
    exclude = input('Any other players? ').lower()

# Prompt user for teams to exclude from the lineup
exclude_team = input('Do you want to exclude any teams from lineups? ').lower()
while exclude_team == 'y':
    team = input('Which team? ')
    exclude_teams.append(team)
    exclude_team = input('Any other teams? ').lower()

if site_for_pred == 'FD':
    cap = 60000
else:
    cap = 50000

# Define the lineup generation function with a salary cap parameter and lock/exclude lists
def generate_lineup(df, salary_cap, excluded_players=exclude_list, locked_players=lock_list, excluded_teams=exclude_teams):
    if excluded_players is None:
        excluded_players = []
    if locked_players is None:
        locked_players = []
    if excluded_teams is None:
        excluded_teams = []

    # Create the pulp problem
    prob = pulp.LpProblem('NFL_weekly', pulp.LpMaximize)

    # Create variables for each player indicating whether they are included in the lineup
    player_vars = [pulp.LpVariable(f'player_{row.name}', cat='Binary') for row in df.itertuples()]

    # Total assigned players constraint
    prob += pulp.lpSum(player_var for player_var in player_vars) == 9

    # Total salary constraint using the provided salary cap
    prob += pulp.lpSum(df.salary.iloc[i] * player_vars[i] for i in range(len(df))) <= salary_cap

    # Create a helper function to return the number of players assigned each position
    def get_position_sum(player_vars, df, position):
        return pulp.lpSum([player_vars[i] * (position in df['position'].iloc[i]) for i in range(len(df))])

    # Position constraints: 1 QB, 1 Defense, 1 TE, 2 RBs, 3 WRs, 1 FLEX (RB, WR, or TE)
    prob += get_position_sum(player_vars, df, 'QB') == 1
    prob += get_position_sum(player_vars, df, 'D') == 1
    prob += get_position_sum(player_vars, df, 'TE') == 1
    prob += get_position_sum(player_vars, df, 'RB') >= 2
    prob += get_position_sum(player_vars, df, 'WR') >= 3

    # Objective: Maximize total predicted points
    prob += pulp.lpSum([df.max_pred.iloc[i] * player_vars[i] for i in range(len(df))])

    # Exclude specific players and entire teams
    for i, row in df.iterrows():
        if row['name'] in excluded_players or row['team'] in excluded_teams:
            prob += player_vars[i] == 0  # Prevent this player from being selected

    # Lock specific players into the lineup
    for i, row in df.iterrows():
        if row['name'] in locked_players:
            prob += player_vars[i] == 1  # Force this player to be selected

    # Solve the problem and print the status
    prob.solve()
    print(pulp.LpStatus[prob.status])

    total_salary_used = 0
    mean_pred_points = 0
    lineup = []

    # Gather lineup details
    for i in range(len(df)):
        if player_vars[i].value() == 1:
            row = df.iloc[i]
            lineup.append((row['position'], row['name'], row.team, row.salary, row.median_pred))
            total_salary_used += row.salary
            mean_pred_points += row.median_pred

    mean_pred_points /= 9  # Divide by total players in roster to get a mean

    # Print the lineup details
    for player in lineup:
        print(player)

    print(f"Total salary used: {total_salary_used}")
    print(f"Total predicted points: {mean_pred_points * 9}")  # Multiply mean by 9 to get total points

    return lineup

# Example of how to call the function for DraftKings

generate_lineup(df, cap, exclude_list, lock_list, exclude_teams)

generate = input('Do you want to generate any more lineups? ').lower()
while generate == 'y':
    site_for_pred = input('Which site? ').lower()
    if site_for_pred == 'fd':
        df = pd.read_csv('FD_predictions_' + str(season) + '_' + str(week) + '.csv')
        cap = 60000
    else: 
        df = pd.read_csv('DK_predictions_' + str(season) + '_' + str(week) + '.csv')
        cap = 50000
        
    # Clear lock list
    clear_lock = input('Do you want to clear the lock list? ').lower()
    if clear_lock == 'y':
        lock_list = []
    lock = input('Do you want to add to the lock list? ').lower()
    while lock == 'y':
        player = input('Which player? ')
        lock_list.append(player)
        lock = input('Any other players? ').lower()

    # Clear exclude list
    clear_exclude = input('Do you want to clear the exclude list? ').lower()
    if clear_exclude == 'y':
        exclude_list = []
    exclude = input('Do you want to add to the exclude list? ').lower()
    while exclude == 'y':
        player = input('Which player? ')
        exclude_list.append(player)
        exclude = input('Any other players? ').lower()

    # Clear exclude teams list
    clear_exclude_teams = input('Do you want to clear the exclude teams list? ').lower()
    if clear_exclude_teams == 'y':
        exclude_teams = []
    exclude_team = input('Do you want to add to the exclude teams list? ').lower()
    while exclude_team == 'y':
        team = input('Which team? ')
        exclude_teams.append(team)
        exclude_team = input('Any other teams? ').lower()

    generate_lineup(df, cap, exclude_list, lock_list, exclude_teams)
    generate = input('Do you want to generate any more lineups? ').lower()


Do you want to lock any players into the lineup?  y
Which player?  Stefon Diggs
Any other players?  n
Do you want to exclude any players from lineups?  y
Which player?  Nico Collins
Any other players?  n
Do you want to exclude any teams from lineups?  y
Which team?  ARI
Any other teams?  n


Optimal
('QB', 'Brock Purdy', 'SF', 6200, 21.95856355329524)
('WR', 'Jayden Reed', 'GB', 6500, 16.26492892275094)
('WR', 'Keenan Allen', 'CHI', 5800, 15.750663686913802)
('WR', 'Tank Dell', 'HOU', 5400, 14.908729553222656)
('RB', 'Josh Jacobs', 'GB', 6400, 14.770952819434472)
('WR', 'Stefon Diggs', 'HOU', 6900, 14.39263460807724)
('TE', 'Evan Engram', 'JAX', 4600, 13.102159066661782)
('RB', 'Justice Hill', 'BAL', 5300, 12.06860065460205)
('D', 'DEN', 'DEN', 2900, 9.665218796847142)
Total salary used: 50000
Total predicted points: 132.88245166180533


Do you want to generate any more lineups?  y
Which site?  DK
Do you want to clear the lock list?  y
Do you want to add to the lock list?  n
Do you want to clear the exclude list?  y
Do you want to add to the exclude list?  n
Do you want to clear the exclude teams list?  y
Do you want to add to the exclude teams list?  n


Optimal
('QB', 'Brock Purdy', 'SF', 6200, 21.95856355329524)
('WR', 'Nico Collins', 'HOU', 7700, 19.41884994506836)
('RB', 'Derrick Henry', 'BAL', 7800, 15.931468988433588)
('WR', 'Keenan Allen', 'CHI', 5800, 15.750663686913802)
('WR', 'Tank Dell', 'HOU', 5400, 14.908729553222656)
('TE', 'Evan Engram', 'JAX', 4600, 13.102159066661782)
('WR', 'Darius Slayton', 'NYG', 3900, 12.14576816558838)
('RB', 'Justice Hill', 'BAL', 5300, 12.06860065460205)
('D', 'MIA', 'MIA', 3300, 9.03822265292697)
Total salary used: 50000
Total predicted points: 134.32302626671282


Do you want to generate any more lineups?  y
Which site?  DK
Do you want to clear the lock list?  y
Do you want to add to the lock list?  y
Which player?  Lamar Jackson
Any other players?  n
Do you want to clear the exclude list?  y
Do you want to add to the exclude list?  y
Which player?  Tank Dell
Any other players?  n
Do you want to clear the exclude teams list?  y
Do you want to add to the exclude teams list?  n


Optimal
('QB', 'Lamar Jackson', 'BAL', 7500, 23.33109664916992)
('WR', 'Nico Collins', 'HOU', 7700, 19.41884994506836)
('WR', 'Jayden Reed', 'GB', 6500, 16.26492892275094)
('WR', 'Keenan Allen', 'CHI', 5800, 15.750663686913802)
('RB', 'Nick Chubb', 'CLE', 5800, 13.406634330749512)
('TE', 'Evan Engram', 'JAX', 4600, 13.102159066661782)
('WR', 'Darius Slayton', 'NYG', 3900, 12.14576816558838)
('RB', 'Justice Hill', 'BAL', 5300, 12.06860065460205)
('D', 'DEN', 'DEN', 2900, 9.665218796847142)
Total salary used: 50000
Total predicted points: 135.15392021835189


Do you want to generate any more lineups?  y
Which site?  DK
Do you want to clear the lock list?  n
Do you want to add to the lock list?  y
Which player?  Tyrone Tracy
Any other players?  n
Do you want to clear the exclude list?  n
Do you want to add to the exclude list?  y
Which player?  Jayden Reed
Any other players?  n
Do you want to clear the exclude teams list?  y
Do you want to add to the exclude teams list?  n


Optimal
('QB', 'Lamar Jackson', 'BAL', 7500, 23.33109664916992)
('WR', 'Nico Collins', 'HOU', 7700, 19.41884994506836)
('RB', 'Kyren Williams', 'LAR', 7600, 16.463248857129802)
('WR', 'Keenan Allen', 'CHI', 5800, 15.750663686913802)
('WR', "Wan'Dale Robinson", 'NYG', 5600, 13.925538063049316)
('TE', 'Evan Engram', 'JAX', 4600, 13.102159066661782)
('WR', 'Darius Slayton', 'NYG', 3900, 12.14576816558838)
('D', 'DEN', 'DEN', 2900, 9.665218796847142)
('RB', 'Tyrone Tracy', 'NYG', 4300, 5.6242145325634585)
Total salary used: 49900
Total predicted points: 129.42675776299197


Do you want to generate any more lineups?  n


In [127]:
# df[df['name'] == 'CJ Stroud']

In [17]:
# generate_lineup(df, 50000, exclude)